# Geocoding


### Table of Content

1. [Import Libraries and the dataset](#import)
2. 

#### 1. Import Libraries and the dataset. <a id = 'import'></a>

In [2]:
#Libraies imported
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
import re
from geopandas.tools import geocode
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [3]:
#Dataset imported

updated_file = 'lat_lon_updated_for_real_estate_withOUT_remarks.csv'
real_estate = pd.read_csv(f'../Datasets/{updated_file}')

#Reset Index
real_estate.reset_index(drop=True)

/var/folders/kv/4xjj0g696_lcyc7ms5pdd2t00000gn/T/ipykernel_93269/3843154784.py:4: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  real_estate = pd.read_csv(f'../Datasets/{updated_file}')


,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,...,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Full Address,latitude,longitude
0,0,0,0,0,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,...,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN,"230, Wakelee Ave",41.347520,-73.090560
1,1,1,1,1,20002,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,...,0.5883,Residential,Single Family,NaN,NaN,NaN,NaN,"390, Turnpike Rd",41.917736,-72.207310
2,2,2,2,2,200212,2020,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,...,0.7248,Residential,Condo,NaN,NaN,NaN,NaN,"5, Chestnut Drive",55.927201,-4.163494
3,3,3,3,3,200243,2020,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,...,0.6958,Residential,Single Family,NaN,NaN,NaN,NaN,"111, Northington Drive",41.790468,-72.902562
4,4,4,4,4,200377,2020,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,...,0.5957,Residential,Single Family,NaN,NaN,NaN,NaN,"70, Far Hills Drive",41.797235,-72.805994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535297,535297,535297,997185,997203,190302,2019,11/29/2019,Waterbury,122 WOLCOTT ST,77870.0,...,0.6953,Three Family,Three Family,NaN,NaN,NaN,NaN,"122, Wolcott St",NaN,NaN
535298,535298,535298,997187,997205,190161,2019,03/02/2020,Newington,22 ROCKLEDGE DR,306000.0,...,0.6711,Single Family,Single Family,NaN,NaN,NaN,NaN,"22, Rockledge Dr",NaN,NaN
535299,535299,535299,997192,997210,190129,2019,04/27/2020,Windsor Locks,19 HATHAWAY ST,121450.0,...,0.5783,Single Family,Single Family,NaN,NaN,NaN,NaN,"19, Hathaway St",NaN,NaN
535300,535300,535300,997193,997211,190504,2019,06/03/2020,Middletown,8 BYSTREK DR,203360.0,...,0.7263,Single Family,Single Family,NaN,NaN,NaN,NaN,"8, Bystrek Dr",NaN,NaN


The dataset is imported. 

Because the dataset is continually being updated after retrieving the coordinates from Open Street Map API, the  latest version of the dataset should be imported, not the raw dataset. 

The latest updated dataset can be downloaded here:

In [4]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535302 entries, 0 to 535301
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0.3      535302 non-null  int64  
 1   Unnamed: 0.2      535302 non-null  int64  
 2   Unnamed: 0.1      535302 non-null  int64  
 3   Unnamed: 0        535302 non-null  int64  
 4   Serial Number     535302 non-null  int64  
 5   List Year         535302 non-null  int64  
 6   Date Recorded     535300 non-null  object 
 7   Town              535302 non-null  object 
 8   Address           535299 non-null  object 
 9   Assessed Value    535302 non-null  float64
 10  Sale Amount       535302 non-null  float64
 11  Sales Ratio       535302 non-null  float64
 12  Property Type     314198 non-null  object 
 13  Residential Type  312773 non-null  object 
 14  Non Use Code      0 non-null       float64
 15  Assessor Remarks  0 non-null       float64
 16  OPM remarks       0 

In [8]:
def save_to_csv():
    real_estate.to_csv('../Datasets/lat_lon_updated_for_real_estate_withOUT_remarks.csv')

In [5]:
import time
from geopy.exc import GeocoderServiceError 

# Initialize the geocoder
geolocator = Nominatim(user_agent="bootcamp student")

# Assuming real_estate is defined elsewhere in your code
real_estate = pd.DataFrame(real_estate)

max_retries = 5
retry_delay = 5  # Set the delay in seconds between retries
last_successful_index = None  # Initialize to None
save_interval = 500  # Define the interval for saving the CSV file
processed_count = 0  # Initialize the counter
print_interval = 300

start_index = 114698
retries = 0 


In [10]:

for index, row in real_estate.iloc[start_index:].iterrows():
    processed_count += 1
    if processed_count % print_interval == 0:
        print(index, "th number of row is being worked on")
    if processed_count % save_interval == 0:
        print("Saving the updated .csv file")
        save_to_csv()
    if pd.notnull(row["longitude"]) or pd.notnull(row["longitude"]):
        #print(index, "th row has lat/lon values")
        # Skip to the next row if either Latitude and Longitude are already present
        continue
    address = row["Full Address"]
    retries =0 
    while retries < max_retries:
        try:
            retries +=1
            #print(index, "th row begin geocoding")
            location = geolocator.geocode(address, timeout=3)
            if location is not None:
                if location.latitude and location.longitude:
                    real_estate.at[index, "latitude"] = location.latitude
                    real_estate.at[index, "longitude"] = location.longitude
                    last_successful_index = index  # Update the index of the last successful geocode
                    #print(index, "geocoded")
                else:
                    real_estate.at[index, "latitude"] = 0
                    real_estate.at[index, "longitude"] = 0
                break  # Successful geocode, break out of the loop

        except GeocoderServiceError as e:
            if "Non-successful status code 500" in str(e):
                print("Geocoding service error 500. Retrying...")
                retries += 1
                time.sleep(retry_delay)
            
    else:
        real_estate.at[index, "latitude"] = 0
        real_estate.at[index, "longitude"] = 0

43898 th number of row is being worked on
44198 th number of row is being worked on
Saving the updated .csv file
44498 th number of row is being worked on
44798 th number of row is being worked on
Saving the updated .csv file
45098 th number of row is being worked on
Saving the updated .csv file
45398 th number of row is being worked on
45698 th number of row is being worked on
Saving the updated .csv file
45998 th number of row is being worked on
46298 th number of row is being worked on
Saving the updated .csv file
46598 th number of row is being worked on
Saving the updated .csv file
46898 th number of row is being worked on
47198 th number of row is being worked on
Saving the updated .csv file
47498 th number of row is being worked on
47798 th number of row is being worked on
Saving the updated .csv file
48098 th number of row is being worked on
Saving the updated .csv file
48398 th number of row is being worked on
48698 th number of row is being worked on
Saving the updated .csv f

In [ ]:
real_estate = real_estate.loc[:, ~real_estate.columns.str.contains('Unnamed')]

In [9]:

# Save the final DataFrame to a CSV file
save_to_csv()
